# Annotating Macromolecular Complexes in the Protein Data Bank: Improving the FAIRness of Structure Data

## Disclaimer

This Jupyter notebook is supplementary material for the publication:

**"Annotating Macromolecular Complexes in the Protein Data Bank: Improving the FAIRness of Structure Data"**, Sri A et al., BioRxiv

The code in this notebook reproduces the analysis presented in the publication.

## Background

Macromolecular complexes are crucial functional units in virtually all cellular processes. Their atomic-level understanding is vital to understanding molecular mechanisms and affects applications, such as developing new therapeutics. The Protein Data Bank (PDB) is the central repository for experimentally determined macromolecular structures. However, it can be challenging to find all instances that represent the same assembly in the PDB due to the current PDB annotation practices, which do not include the annotation of assemblies. This study highlights the importance of annotations for macromolecular complexes and the need for more robust methods to identify and classify these complexes across the PDB. We propose a new approach that uses external resources such as the Complex Portal and Gene Ontology to describe assemblies accurately and put them into their biological contexts. We anticipate that the new approach to identifying and classifying complexes will improve the usability and utility of the PDB for researchers in the field of structural biology and drug discovery.

### Primary use case of this Jupyter notebook

This notebook demonstrates how to extract assembly information from the data generated by the weekly-updated complex identifier process of PDBe.
Using this notebook, or by slightly editing the code presented here, it is easy to identify assemblies such as:

* Protein-only assemblies
* Assemblies containing RNA or DNA
* Assemblies with components unmapped to UniProt (e.g. antibodies)

## Importing dependencies

This code imports several modules that are used in the script.

* The `pandas` module is used for data manipulation and analysis
* `utility` is a custom module that contains utility functions used in the script
* `matplotlib` is a plotting library and `pyplot` is a sublibrary within it, often used for creating static, two-dimensional plots.
* `constants` is a custom module that contains constants used in the script. `UNIPROT_PATTERN` and `RFAM_PATTERN` are constants that are defined in the constants module and used in the script.

In [219]:
import pandas as pd
import numpy as np
import scripts.utility as ui
from matplotlib import pyplot as plt
from scripts.constants import UNIPROT_PATTERN, RFAM_PATTERN, RIBOSOMES_RFAM_MAPPING

## Pre-Processing

This section contains some pre-processing that we need to do before analysing the data

In [220]:
# Read the CSV file containing the data on assemblies
df = pd.read_csv("data/assemblies_data.csv")
prefered_assemblies_dict = dict(zip(df['ASSEMBLIES'], df['PREFERED_ASSEMBLIES']))

In [221]:
# This method generates the preferred assemblies string
def get_prefered_assembly_str(assembly_str):
    prefered_assemblies = []
    assemblies = assembly_str.split(",")
    for assembly in assemblies:
        prefered_assembly = prefered_assemblies_dict.get(assembly)
        if prefered_assembly is True:
            prefered_assemblies.append(assembly)
    return ",".join(prefered_assemblies)

In [222]:
# Group the rows in dataframe based on the assembly string
df = df.astype(str).groupby('ASSEMBLY_STRING', sort=False).agg(lambda x: ','.join(x.unique())).reset_index()
# Drop the existing PREFERED_ASSEMBLIES col in the dataframe
df = df.drop('PREFERED_ASSEMBLIES', axis=1)

In [223]:
# Create a new column that shows the prefered assemblies string for each unique assembly composition
df["PREFERED_ASSEMBLIES"] = df["ASSEMBLIES"].apply(lambda x: get_prefered_assembly_str(x))

In [224]:
# Create a new column that indicates whether each unique assembly composition contains chimeric chains
df["hasChimericChains"] = df["ASSEMBLY_STRING"].apply(lambda x: ui.check_for_chimeric_chains(x))
# Exclude unique assemblies that contain chimeric chains
df = df[df["hasChimericChains"] == False]

## Assembly Classification

We can classify the assemblies into three types:
* monomeric
* homomeric
* heteromeric

A monomeric assembly is defined as an assembly that consists of a single subunit. Homomeric assemblies are formed by repeated units of a single subunit while heteromeric assemblies are composed of multiple distinct subunits. 

In [225]:
df["ASSEMBLY_TYPE"] = df["ASSEMBLY_STRING"].apply(lambda x: ui.get_assembly_type(x))

## 1) Assemblies analysis

This section contains some general analyses that were carried out on assemblies across the whole PDB archieve in order to learn their numbers, composition of the assemblies and how many components in the assemblies are mapped to external resources such as [UniProt](https://www.uniprot.org/) for protein entities and [Rfam](https://rfam.org/) for RNA entities

### 1.2) Basic statistics

#### 1.2.1) Unique assemblies

First, we check how many unique assembly compositions are there in the PDB.

In [328]:
print("There are %i unique assembly compositions in the Protein Data Bank." % len(df))

In [227]:
# Column to indicate the exp methods used to solved the structures of each unique assembly composition
df["EXP_METHOD"] = df["ASSEMBLIES"].apply(lambda x: ui.get_exp_methods(x))
exp_methods = df["EXP_METHOD"].value_counts().head(20)
exp_methods

In [228]:
# Column to indicate the species name for each unique assembly composition
df["SPECIES_NAME"] = df["ASSEMBLY_STRING"].apply(lambda x: ui.get_species_name(x))
df["SPECIES_NAME"].value_counts().head(10)

#### 1.2.2) Unique assemblies containing proteins

Next, we look at how many of these unique assemblies contain proteins.

In [229]:
# Column to indicate the number of assemblies each unique assembly has
df["NUM_ASSEMBLIES"] = df["ASSEMBLIES"].apply(lambda x: ui.count_assemblies(x))

# Column to indicate the number of distinct PDBIDs each unique assembly has
df["NUM_PDB"] = df["ASSEMBLIES"].apply(lambda x: ui.count_unique_pdb(x))

# Column to indicate whether each unique assembly has a protein component
df["hasProteinComponent"] = df["ASSEMBLY_STRING"].apply(lambda x: ui.check_for_protein(x))

df[df["hasProteinComponent"] == True].head()

In [230]:
print("There are %i unique assemblies containing protein components." % len(df[df["hasProteinComponent"] == True]))

#### 1.2.3) Unique assemblies containing RNAs

We can also count assemblies that contain RNA molecules.

In [231]:
# Column to indicate whether each unique assembly has a RNA component
df["hasRNAComponent"] = df["ASSEMBLY_STRING"].apply(lambda x: ui.check_for_rna(x))

df[df["hasRNAComponent"] == True]

In [232]:
print("There %i unique assemblies containing RNA components." % len(df[df["hasRNAComponent"] == True]))

#### 1.2.4) Unique assemblies containing DNAs

Finally, we count the assemblies which contain DNA molecules.

In [233]:
# Column to indicate whether each unique assembly has a DNA component
df["hasDNAComponent"] = df["ASSEMBLY_STRING"].apply(lambda x: ui.check_for_dna(x))

df[df["hasDNAComponent"] == True].head()

In [234]:
print("There %i unique assemblies containing DNA components." % len(df[df["hasDNAComponent"] == True]))

### 1.2.5) Summarising what we found so far

Let's summarise how many assemblies fall into specific categories based on their compositions.

In [235]:
"""
Column to indicate the type of polymer present in each unique assembly. The polymer can either  be protein, 
DNA, or RNA
"""
df["ASSEMBLY_POLYMERS"] = df.apply(lambda row: ui.assembly_composition(row["hasProteinComponent"], row["hasRNAComponent"], row["hasDNAComponent"]), axis=1)

In [236]:
assemblies_polymer_composition = df["ASSEMBLY_POLYMERS"].value_counts()
assemblies_polymer_composition

Let's visualise the data using a bar chart.

In [237]:
ax = assemblies_polymer_composition.plot(kind="bar")
ax.set_title("Assemblies polymer composition in PDB")

We can see that the vast majority of assemblies in the PDB are protein assemblies followed by DNA-protein and RNA-protein assemblies respectively.

## 1.3) Focusing on protein assemblies

Next, we will investigate some statistics of protein assemblies.

### 1.3.1) Creating protein-only assembly subset

We start by creating a data frame for protein assemblies.


In [238]:
# Create a new dataframe to represent protein assemblies
df_protein = df[df["ASSEMBLY_POLYMERS"] == "protein"].copy()
df_protein.head()

### 1.3.2) Assembly components mapped to UniProt accessions

Then, we check if the components in the protein assemblies are all mapped to UniProt accessions. Reasons why a protein may not be mapped to a UniProt accession include:

* Length limitation (sequence too short to map reliably)
* Type limitation (certain classes of proteins, such as antibodies, have no UniProt accessions

In [239]:
# Column to indicate whether all components in the protein assembly are mapped to UniProt
df_protein["hasAllUNPMapping"] = df_protein["ASSEMBLY_STRING"].apply(lambda x: ui.validate_uniprot(x, all))

In [240]:
# Number of protein assemblies with all components mapped to UniProt
mapped_protein_assemblies_all = df_protein[df_protein["hasAllUNPMapping"] == True]
mapped_protein_assemblies_all.head()

#### 1.3.2.1) Every component mapped to UniProt accessions

In [241]:
# Calculate the percentage of protein assemblies with all components mapped to UniProt
print("Across the PDB, %i%% percent of protein assemblies have all of their components mapped to UniProt" % (len(mapped_protein_assemblies_all)/len(df_protein)*100))

1.3.2.2) Any of the components mapped to UniProt accessions

In [242]:
# Column to indicate whether any one of the components in the protein assembly is mapped to UniProt
df_protein["hasAnyUNPMapping"] = df_protein["ASSEMBLY_STRING"].apply(lambda x: ui.validate_uniprot(x, any))

In [243]:
# Number of protein assemblies with at least one component mapped to UniProt
mapped_protein_assemblies_any = df_protein[df_protein["hasAnyUNPMapping"] == True]
mapped_protein_assemblies_any.head(10)

In [244]:
# Calculate the percentage of protein assemblies with at least one component mapped to UniProt
print("Across the PDB archive %i%% of the protein-only assemblies have at least one component mapped to UniProt" % (len(mapped_protein_assemblies_any)/len(df_protein)*100))

## 1.4) Focusing on RNA assemblies

Next, we will investigate some statistics of RNA assemblies.

### 1.4.1) Creating RNA-only assembly subset

We start by creating a data frame for RNA assemblies.

In [245]:
# Create a new dataframe to represent protein assemblies
df_rna = df[df["ASSEMBLY_POLYMERS"] == "RNA"].copy()
df_rna

### 1.3.2) Assembly components mapped to Rfam accessions
Then, we check if the components in the RNA assemblies are all mapped to Rfam accessions. Reasons why an RNA may not be mapped to a Rfam accession include:

Length limitation (sequence too short to map reliably)
Type limitation (certain types, such as mRNAs, have no Rfam accessions

In [246]:
# Column to indicate whether all components in the RNA assembly are mapped to Rfam
df_rna["hasAllRfamMapping"] = df_rna["ASSEMBLY_STRING"].apply(lambda x: ui.validate_rfam(x, all))

In [247]:
# Number of rna assemblies with all components mapped to Rfam
mapped_rna_assemblies_all = df_rna[df_rna["hasAllRfamMapping"] == True]
mapped_rna_assemblies_all.shape[0]

## 2) Variation of stoichiometry in assemblies in the PDB

Structures in the PDB archive are determined to investigate a specific scientific problem. Consequently, a PDB structure does not always represent the biological complex. For example, PDB entry [1abw](https://www.ebi.ac.uk/pdbe/entry/pdb/1abw) contains a human haemoglobin trimer where two copies of the alpha subunit are linked together. This trimer represents a deviation from the biological heterotetrameric state of haemoglobin. In this section of the notebook, we analyse UniProt accessions in which the assemblies exist with different stoichiometries.

In [248]:
# Create a dataframe to represent monomeric protein assemblies
df_protein_monomeric = df_protein[df_protein["ASSEMBLY_TYPE"] == "monomeric"]
# Create a dataframe to represent homomeric protein assemblies
df_protein_homomeric = df_protein[df_protein["ASSEMBLY_TYPE"] == "homomeric"]
# Create a dataframe to represent heteromeric protein assemblies
df_protein_heteromeric = df_protein[df_protein["ASSEMBLY_TYPE"] == "heteromeric"]

# Calculate the percentage of monomeric protein assemblies
percentage_monomeric_assemblies = (len(df_protein_monomeric)/len(df_protein))*100
# Calculate the percentage of homomeric protein assemblies
percentage_homomeric_assemblies = (len(df_protein_homomeric)/len(df_protein))*100
# Calculate the percentage of heteromeric protein assemblies
percentage_heteromeric_assemblies = (len(df_protein_heteromeric)/len(df_protein))*100

In [249]:
print("Out of all the assemblies in the PDB, %i%% are monomeric, %i%% are homomeric and %i%% are heteromeric." % (
    percentage_monomeric_assemblies,
    percentage_homomeric_assemblies,
    percentage_heteromeric_assemblies
))

### 2.1) Homomeric protein assemblies

Next, we will investigate the homomeric protein assemblies.

In [250]:
# Create a dataframe to represent homomeric protein assemblies with all components mapped to UniProt
df_protein_homomeric_mapped = df_protein_homomeric[(df_protein_homomeric["hasAllUNPMapping"] == True)]
df_protein_homomeric_mapped

Show the top 10 species of the homomeric assemblies

In [251]:
homomeric_species_count = df_protein_homomeric_mapped["SPECIES_NAME"].value_counts().head(10)
homomeric_species_count

In [252]:
hx = homomeric_species_count.plot(kind="bar")
hx.set_title("Top 10 species from which homomeric assemblies are solved")

In [253]:
# Calculate the percentage of homomeric protein assemblies that have all components mapped to UniProt
percentage_homomeric_assemblies_mapped = (len(df_protein_homomeric_mapped)/len(df_protein_homomeric))*100

In [254]:
print("%i%% of the homomeric protein assemblies have all their components mapped to UniProt accession." % percentage_homomeric_assemblies_mapped)

We then create a list of assembly composition strings, and group them by UniProt accessions.

In [255]:
# Create a list of UniProt accessions with stoichiometry in the homomeric assemblies
homomeric_assembly_strings = df_protein_homomeric_mapped["ASSEMBLY_STRING"].tolist()

In [256]:
# Group the UniProt accessions in homomeric assemblies
homomeric_grouped_UNP_accessions = ui.group_UniProt_accessions(homomeric_assembly_strings)
print ("There are %i groups." % len(homomeric_grouped_UNP_accessions))

Next, we keep only those groups that has more than one assembly states.

In [257]:
# Only keep UniProt accessions that display 2 or more stoichiometry number
homomeric_UNP_accessions_with_muliple_stoic = {k: sorted([int(x) for x in grp]) for k, grp in homomeric_grouped_UNP_accessions.items() if len(grp) > 1}
# Number of UniProt accessions with multiple stoichiometry numbers
homomeric_UNP_accession_stoic_count = len(homomeric_UNP_accessions_with_muliple_stoic)

In [258]:
print("There are %i UniProt accessions that exist in different stoichiometries in homomeric assemblies" % homomeric_UNP_accession_stoic_count)

In [259]:
# Count the stoichiometry number for UniProt accessions with two or more stoichiometries
homomeric_UNP_accessions_with_multiple_stoic_count = [(k, len(grp)) for k, grp in homomeric_UNP_accessions_with_muliple_stoic.items()]

In [260]:
# Sort the count of stoichiometry number in descending order
sorted_homomeric_UNP_accessions_with_multiple_stoic_count = sorted(homomeric_UNP_accessions_with_multiple_stoic_count, key=lambda x: x[1], reverse=True)

In [261]:
# Show the top 10 UniProt accessions with highest stoichiometry number count
sorted_homomeric_UNP_accessions_with_multiple_stoic_count[:12]

### 2.2) Heteromeric protein assemblies

Now, we will investigate the heteromeric protein assemblies.

In [262]:
# Create a dataframe to represent heteromeric protein assemblies with all components mapped to UniProt
df_protein_heteromeric_mapped = df_protein_heteromeric[(df_protein_heteromeric["hasAllUNPMapping"] == True)]
df_protein_heteromeric_mapped.head()

In [263]:
# Calculate the percentage of heteromeric protein assemblies that have all components mapped to UniProt
percentage_heteromeric_assemblies_mapped = (len(df_protein_heteromeric_mapped)/len(df_protein_heteromeric))*100

In [264]:
print("%i%% of the heteromeric protein assemblies have all their components mapped to UniProt accession." % percentage_heteromeric_assemblies_mapped)

In [265]:
# Create a list of UniProt accessions with stoichiometry in the heteromeric assemblies
heteromeric_assembly_strings = df_protein_heteromeric_mapped["ASSEMBLY_STRING"].tolist()
heteromeric_assembly_components = [assembly.split(",") for assembly in heteromeric_assembly_strings]
heteromeric_assembly_components = [item for sublist in heteromeric_assembly_components for item in sublist] 
heteromeric_assembly_components = list(set(heteromeric_assembly_components))

In [266]:
# Group the UniProt accessions in the heteromeric assemblies
heteromeric_grouped_UNP_accessions = ui.group_UniProt_accessions(heteromeric_assembly_components)
print ("There are %i groups." % len(heteromeric_grouped_UNP_accessions))

Next, we keep only those groups with more than one assembly states.

In [267]:
# Only keep UniProt accessions that display 2 or more stoichiometry number
heteromeric_UNP_accessions_with_muliple_stoic = {k: sorted([int(x) for x in grp]) for k, grp in heteromeric_grouped_UNP_accessions.items() if len(grp) > 1}
# Number of UniProt accessions with multiple stoichiometry numbers
heteromeric_UNP_accession_stoic_count = len(heteromeric_UNP_accessions_with_muliple_stoic)

In [268]:
print("There are %i UniProt accessions that exist in different stoichiometries in heteromeric assemblies" % heteromeric_UNP_accession_stoic_count)

In [269]:
# Count the stoichiometry number for UniProt accessions with two or more stoichiometries
heteromeric_UNP_accessions_with_multiple_stoic_count = [(k, len(grp)) for k, grp in heteromeric_UNP_accessions_with_muliple_stoic.items()]

In [270]:
# Sort the count of stoichiometry number in descending order
sorted_heteromeric_UNP_accessions_with_multiple_stoic_count = sorted(heteromeric_UNP_accessions_with_multiple_stoic_count, key=lambda x: x[1], reverse=True)

In [271]:
# Show the top 10 UniProt accessions with highest stoichiometry number count
sorted_heteromeric_UNP_accessions_with_multiple_stoic_count[:12]

We now compare the UniProt accessions that are used by homomeric and heteromeric assemblies

In [272]:
added, removed, modified, same = ui.dict_compare(homomeric_UNP_accessions_with_muliple_stoic, heteromeric_UNP_accessions_with_muliple_stoic)

In [273]:
print("There are %i UniProt accessions unique for homomeric assemblies" % len(added))

In [274]:
print("There are %i UniProt accessions unique for heteromeric assemblies" % len(removed))

In [275]:
print("There are %i UniProt accessions that are common to both homomeric and heteromeric assemblies with the same assembly states" % len(same))

In [276]:
print("There are %i UniProt accessions that are common to both homomeric and heteromeric assemblies with varying assembly states" % len(modified))

## 3) Sub-assembly analysis

A biological assembly can consist of multiple components where each component can have varying stoichiometry. This is best illustrated by the ribosome which is the molecular machine responsible for protein synthesis in every living cell. The ribosome is made up of two ribosomal subunits (small and large ribosomal subunits) and can bind multiple tRNAs (1-3 tRNAs), mRNA, and diverse protein factors. The binding of tRNAs and the various protein factors can induce the ribosome to undergo large-scale conformational changes which affects how the ribosome functions. As such, the ability to identify the sub- and super-assemblies of a biological assembly is key in identifying the contribution of each component to the function of the whole assembly and the relationships that exist between the components.

We define sub- and super-assembly using the composition and stoichiometry data. We defined a sub-assembly as an assembly where all components map to UniProt accessions and where these components are also present in another assembly. For example, PDB entry [5jdo](https://pdbe.org/5jdo) contains haemoglobin (UniProt [P68871](https://pdbe-kb.org/proteins/P68871) and UniProt [P69905](https://pdbe-kb.org/proteins/P69905)) bound to the haptoglobin-haemoglobin receptor (UniProt [G0UVW6](https://pdbe-kb.org/proteins/G0UVW6)). A sub-assembly of this assembly is haemoglobin. We defined a super-assembly as an assembly that has all the same elements of another assembly plus additional members. Again, all members must map to a UniProt accession. In the example above,  PDB entry [5jdo](https://pdbe.org/5jdo is a super-assembly of haemoglobin.

### 3.1) Reading in the data

In [277]:
df_subassemblies = pd.read_csv("data/subassemblies_data.csv")
df_subassemblies.head()

In [278]:
print("There are %i unique sub-assemblies, and %i unique super-assemblies in the PDB." % (
    df_subassemblies["SUBASSEMBLY_ID"].nunique(),
    df_subassemblies["SUPERASSEMBLY_ID"].nunique()
))

### 3.2) Matching sub- and super-assemblies

We can match sub- and super-assemblies.

In [279]:
subassembly_list = df_subassemblies["SUBASSEMBLY_ID"].tolist()
superassembly_list = df_subassemblies["SUPERASSEMBLY_ID"].tolist()
subassembly_pairs = list(zip(subassembly_list, superassembly_list))

In [280]:
subassembly_pairs[:10]

In [281]:
grouped_subassemblies, grouped_superassemblies = ui.group_subassemblies(subassembly_pairs)

In [282]:
# Assemblies having at least two super-assemblies
assemblies_with_muliple_superassembly = {k: grp for k, grp in grouped_subassemblies.items() if len(grp) > 1}
print("Only %i assemblies have 2 or more super-assemblies" % len(assemblies_with_muliple_superassembly))

In [283]:
# Assemblies having at least two super-assemblies
assemblies_with_muliple_subassembly = {k: grp for k, grp in grouped_superassemblies.items() if len(grp) > 1}
print("Only %i assemblies have 2 or more sub-assemblies" % len(assemblies_with_muliple_subassembly))

### 3.3) Finding the assemblies in the PDB which have the highest number of super-assemblies

In [284]:
# Count the number of superassemblies
superassemblies_count = [(k, len(grp)) for k, grp in assemblies_with_muliple_superassembly.items()]

# Sort the count of superassemblies in descending order
sorted_superassemblies_count = sorted(superassemblies_count, key=lambda x: x[1], reverse=True)

# Show the top 10 entries with the highest number of superassemblies
sorted_superassemblies_count[:10]

# Get the PDB Complex ID for the top 10 entries
top_ten_entries_ids = [x[0] for x in sorted_superassemblies_count[:10]]

top_ten_entries_ids

# Show the assemblies data for the top 10 entries
df[df["PDB_COMPLEX_ID"].isin(top_ten_entries_ids)]

# 4) Naming of Assemblies

Providing a good name for a complex in the PDB poses a significant challenge. Complexes with a singular component can get the name of this component, regardless of the stoichiometry of the component. Examples of this would be Lysozyme, Insulin or Ferritin. In assemblies where the singular component is either DNA or RNA, and no mapping to external resources is present, the name is given as DNA or RNA, respectively. In the case of proteins, we can use the protein name the depositor provided if no external mapping is available. If the component maps to a UniProt accession, then we can use the protein name from UniProt. This approach allows us to automatically name some of the multi-component assemblies, including proteins bound to nucleic acids.


In [285]:
pdb_complexes = df[df["ASSEMBLY_TYPE"] != "monomeric"]
pdb_complexes

## 4.1) Count the number of assemblies that have a name


In [286]:
sum_named_assemblies = (df['ASSEMBLY_NAME'] != 'nan').sum()
percentage_named_assemblies = sum_named_assemblies/df.shape[0]*100

In [287]:
print("Sum of assemblies with name: {}".format(sum_named_assemblies))

In [288]:
print("Percentage of assemblies with name: {:.1f}%".format(percentage_named_assemblies))

## 4.2) Count the number of names generated from each naming category

In [289]:
df["NAME_SOURCE"].value_counts()

We can see that majority of the names are derived from UniProt

## 4.3) Count the total number of names derived from UniProt

### 4.31) Breakdown the number of names derived from each UniProt category

In [290]:
up = df[df["NAME_SOURCE"].str.contains("UniProt", na=False)]
print(up.groupby("NAME_SOURCE").size().sort_values(ascending=False))

In [291]:
# Num of unique assemblies 
up.shape[0]

### 4.32) Count the number of PDB entries that have names derived from UniProt

In [292]:
df.loc[df["NAME_SOURCE"].str.contains("UniProt", na=False), 'NUM_PDB'].sum()

## 4.4) Count the total number of names derived from Complex Portal

In [293]:
# Num of unique assemblies 
df["NAME_SOURCE"].str.contains("complex portal").sum()

### 4.41) Breakdown the number of names derived from each Complex Portal category

In [294]:
assemblies_with_ComplexPortal_names = df[df["NAME_SOURCE"].str.contains("complex portal", na=False)]
print(assemblies_with_ComplexPortal_names.groupby("NAME_SOURCE").size().sort_values(ascending=False))

### 4.42) Count the number of PDB entries that have names derived from Complex Portal

In [295]:
df.loc[df["NAME_SOURCE"].str.contains("complex portal", na=False), 'NUM_PDB'].sum()

## 4.5) Count the number of unique assemblies & PDB entries that are named as heterodimer

In [296]:
condition = df["NAME_SOURCE"] == "heterodimer"
hd = df[condition]
# Num of unique assemblies
hd.shape[0]

In [297]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.6) Count the number of unique assemblies & PDB entries that are named as ribosomes

In [298]:
condition = df["NAME_SOURCE"].str.contains("ribosome")
ribosome = df[condition]
# Num of unique assemblies
ribosome.shape[0]

In [299]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.7) Count the number of unique assemblies & PDB entries that have names derived from Gene Ontology

In [300]:
condition = df["NAME_SOURCE"] == "Gene Ontology"
go = df[condition]
# Num of unique assemblies
go.shape[0]

In [301]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

In [302]:
go["ASSEMBLY_NAME"].value_counts()

## 4.8) Count the number of unique assemblies & PDB entries that have manually curated names

In [303]:
condition = df["NAME_SOURCE"] == "PDBe curated"
mc = df[condition]
# Num of unique assemblies
mc.shape[0]

In [304]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.9) Count the number of unique assemblies & PDB entries that have general nucleic acid names

In [305]:
condition = df["NAME_SOURCE"] == "general nucleic acid name"
gn = df[condition]
# Num of unique assemblies
gn.shape[0]

In [306]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.10) Count the number of unique assemblies & PDB entries that have common entity names

In [307]:
condition = df["NAME_SOURCE"] == "common name from entity names"
en = df[condition]
# Num of unique assemblies
en.shape[0]

In [308]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.11) Count the number of unique assemblies & PDB entries that have all antibodies names

In [309]:
condition = df["NAME_SOURCE"] == "all_antibodies"
ab = df[condition]
# Num of unique assemblies
ab.shape[0]

In [310]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.12) Count the number of unique assemblies & PDB entries that have all unmapped protein names

In [311]:
condition = df["NAME_SOURCE"] == "all_unmapped_proteins_without_antibodies"
aup = df[condition]
# Num of unique assemblies
aup.shape[0]

In [312]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

## 4.13) Count the number of PDB entries that have Rfam names

In [313]:
condition = df["NAME_SOURCE"].str.contains("RNA name from Rfam")
rf = df[condition]
# Num of unique assemblies
rf.shape[0]

In [314]:
# Num of PDB entries
df.loc[condition, 'NUM_PDB'].sum()

# 5) Symmetry analysis

PDB assemblies represent different forms that certain complex molecules, like proteins, can take. For example, a protein called haemoglobin is known to exist in two different states, one called the relaxed state, or R state, and the other called the tense state, or T state. These two states are structurally distinct from each other, meaning they have different shapes and forms. In order to gain a better understanding of these different shapes and forms, we used a computer program called [AnAnaS](https://team.inria.fr/nano-d/software/ananas/), which stands for "Analyse and Annotate Structures". This program is used to study the symmetry of different PDB assemblies, which can provide insights into the conformational heterogeneity of these complex molecules.

## 5.1) Load symmetry data

First we add several columns to the protein assembly data set:

* Symmetry operators
* A flag for consistent symmetry across various instances of the same assembly
* The number of distinct symmetry operator each unique assembly composition has

In [315]:
# Add a column to indicate the symmetry operators of the assemblies for each unique assembly composition
df_protein["SYMMETRY_OPS"] = df_protein["PREFERED_ASSEMBLIES"].apply(lambda x: ui.get_sym_op(x))

# Add a column to indicate whether all assemblies for each unique assembly composition have consistent symmetry
df_protein["CONSISTENT_SYMMETRY"] = df_protein["SYMMETRY_OPS"].apply(lambda x: ui.validate_consistent_symmetry(x))

# Add a column to indicate the number of distinct symmetry operators each unique assembly composition has
df_protein["SYM_NUM"] = df_protein["SYMMETRY_OPS"].apply(lambda x: ui.count_unique_symmetry(x))

# Add a column to show all the unique symmetries for each unique assembly composition
df_protein["ALL_SYM"] = df_protein["SYMMETRY_OPS"].apply(lambda x: ui.get_unique_symmetries(x))

# Add a column to show the most frequent symmetry for each unique assembly composition
df_protein["PRIMARY_SYM"] = df_protein["SYMMETRY_OPS"].apply(lambda x: ui.most_frequent_symmetry(x))

## 5.2) Symmetry statistics

In [316]:
df_crystal = df['EXP_METHOD'] == 'X-ray'
df_consistent = df_protein[(df_protein['CONSISTENT_SYMMETRY'] == True) & (df_protein['PRIMARY_SYM'] != "no-sym") & df_crystal]

In [317]:
print(f"There are {df_consistent.shape[0]} unique assemblies that have a symmetry")

In [318]:
cyclic_assemblies = df_consistent[df_consistent['PRIMARY_SYM'].str.contains("c")]
dihedral_assemblies = df_consistent[df_consistent['PRIMARY_SYM'].str.contains("d")]
tetrahedral_assemblies = df_consistent[df_consistent['PRIMARY_SYM'] == "t"]
octahedral_assemblies = df_consistent[df_consistent['PRIMARY_SYM'] == "o"]
icosahedral_assemblies = df_consistent[df_consistent['PRIMARY_SYM'] == "i"]

In [319]:
print(f"There are {cyclic_assemblies.shape[0]} cyclic assemblies, {dihedral_assemblies.shape[0]} dihedral assemblies, {tetrahedral_assemblies.shape[0]} tetrahedral assemblies, {octahedral_assemblies.shape[0]} octahedral assemblies and finally, {icosahedral_assemblies.shape[0]} icosahedral assemblies")

In [320]:
# Show all the cyclic assemblies variants
cyclic_assemblies_variants = cyclic_assemblies['PRIMARY_SYM'].value_counts()
cyclic_assemblies_variants

In [321]:
cx = cyclic_assemblies_variants.plot(kind="bar")
cx.spines['top'].set_visible(False)
cx.spines['right'].set_visible(False)
cx.set_yscale('log')
cx.set_xticklabels(cx.get_xticklabels(), rotation=0)

In [322]:
# Show all the dihedral assemblies variants
dihedral_assemblies_variants = dihedral_assemblies['PRIMARY_SYM'].value_counts()
dihedral_assemblies_variants

In [323]:
dx = dihedral_assemblies_variants.plot(kind="bar")
# dx.set_title("Dihedral (Dn) symmetry groups detected")
dx.spines['top'].set_visible(False)
dx.spines['right'].set_visible(False)
dx.set_yscale('log')
dx.set_xticklabels(dx.get_xticklabels(), rotation=0)

## 5.3) Listing assemblies with inconsistent symmetries

We can list assemblies where the symmetry data is inconsistent between different instances of the same assembly. This can highlight potential annotation errors, or genuinely interesting assemblies.

In [324]:
df_inconsistent_symmetry = df_protein[df_protein['CONSISTENT_SYMMETRY'] == False]
# Number of unique assemblies containing members with inconsistent symmetry
df_inconsistent_symmetry.shape[0]

In [325]:
# df2 = df_inconsistent_symmetry.copy()
# df2["ASYM_ASSEMBLIES"] = df2.apply(lambda row: ui.get_asymmetrical_assemblies(row["PREFERED_ASSEMBLIES"], row["SYMMETRY_OPS"]), axis=1)
# df2["EXTENDED_SYMMETRY_OPS"] = df2.apply(lambda row: ui.get_extended_symmetry_operators(row["PREFERED_ASSEMBLIES"], row["SYMMETRY_OPS"]), axis=1)

In [326]:
# df3 = df2[["ASSEMBLY_STRING", "PDB_COMPLEX_ID", "ASSEMBLY_NAME", "PRIMARY_SYM", "ALL_SYM", "ASYM_ASSEMBLIES", "EXTENDED_SYMMETRY_OPS"]]
# df3.reset_index()
# df3.to_csv("inconsistent_symmetry.csv", index=False)

We can also investigate the data by explicitly excluding asymmetric C1 instance.

In [327]:
# Assemblies with inconsistent symmetry operators (excluding asymmetric C1)
df_protein[(df_protein["CONSISTENT_SYMMETRY"] == False) & (~df_protein["SYMMETRY_OPS"].str.contains("no-sym"))]

## Conclusion

This work provides an archive-wide analysis of assemblies in the PDB archive and demonstrates the power of mapping individual components of assemblies to external databases. Our new approach allows the otherwise complex aggregation of assemblies. With the help of mappings to external databases, it also became possible to name over 80% of assemblies in the PDB, representing over 90% of PDB entries. These names allow users of PDBe to find complexes that previously could only be found by relying on the title of the PDB entry or listing all the complex components. Searching by title or listing components is likely to lead to incomplete and inconsistent search results due to the heterogeneity of assemblies in the PDB archive. We are making the complexes name list and mapping data files available to the community to aid in analysing and identifying macromolecular complexes.

## Authors

* Sri Appasamy
* Mihaly Varadi

### Last updated March 2023